In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
online_retail = pd.read_excel("C:/Users/navee/OneDrive/Desktop/GitHub/MLprojects/Recommendation_system_online_retail_store/Online_Retail.xlsx")

online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France


### For market basket analysis, we need data on the level of trannsactions

In [3]:
online_retail = online_retail[online_retail.Quantity > 0]

In [4]:
online_retail = online_retail[online_retail.Description != "POSTAGE"]

In [5]:
online_retail = online_retail.reset_index().drop(columns='index')

In [6]:
online_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18511 entries, 0 to 18510
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    18511 non-null  object        
 1   StockCode    18511 non-null  object        
 2   Description  18511 non-null  object        
 3   Quantity     18511 non-null  int64         
 4   InvoiceDate  18511 non-null  datetime64[ns]
 5   UnitPrice    18511 non-null  float64       
 6   CustomerID   18445 non-null  float64       
 7   Country      18511 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.1+ MB


In [7]:
online_retail.isna().sum()

InvoiceNo       0
StockCode       0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID     66
Country         0
dtype: int64

In [8]:
online_retail.StockCode.nunique()

2112

In [9]:
online_retail.Description.nunique()

2155

In [10]:
online_retail['item'] = online_retail.StockCode.astype(str) + "_" + online_retail.Description

In [11]:
online_retail.item.nunique()

2158

In [12]:
print("Total unique countries where products are being sold : ",online_retail.Country.nunique())

online_retail.Country.value_counts()

Total unique countries where products are being sold :  6


Country
Germany      8668
France       8108
Australia    1184
Singapore     222
USA           179
Canada        150
Name: count, dtype: int64

# we have to do the market basket analysis on the Country to Country basis

# 1. For Germany

In [13]:
retail_germany = online_retail[online_retail.Country == "Germany"]

retail_germany = retail_germany.reset_index().drop(columns='index')

retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany,22809_SET OF 6 T-LIGHTS SANTA
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany,84347_ROTATING SILVER ANGELS T-LIGHT HLDR
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany,84945_MULTI COLOUR SILVER T-LIGHT HOLDER
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany,22242_5 HOOK HANGER MAGIC TOADSTOOL
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany,22244_3 HOOK HANGER MAGIC GARDEN


In [14]:
retail_germany = retail_germany.pivot_table(index='InvoiceNo', columns='item', values='CustomerID', aggfunc='count' ).fillna(0)


retail_germany.head()

item,10002_INFLATABLE POLITICAL GLOBE,10125_MINI FUNKY DESIGN TAPES,10135_COLOURING PENCILS BROWN TUBE,11001_ASSTD DESIGN RACING CAR PEN,15034_PAPER POCKET TRAVELING FAN,15036_ASSORTED COLOURS SILK FAN,15039_SANDALWOOD FAN,15044A_PINK PAPER PARASOL,15044B_BLUE PAPER PARASOL,15044D_RED PAPER PARASOL,...,90161D_ANT COPPER PINK BOUDICCA BRACELET,90170_DAISY HAIR BAND,90173_PAIR BUTTERFLY HAIR CLIPS,90201A_PURPLE ENAMEL FLOWER RING,90201B_BLACK ENAMEL FLOWER RING,90201C_RED ENAMEL FLOWER RING,90201D_GREEN ENAMEL FLOWER RING,90202D_PINK ENAMEL FLOWER HAIR TIE,90204_PAIR OF ENAMEL BUTTERFLY HAIRCLIP,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def encode(x):
    if x <= 0:
        return 0
    else:
        return 1

In [16]:
retail_germany = retail_germany.applymap(encode)


retail_germany.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\4035776250.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_germany = retail_germany.applymap(encode)


item,10002_INFLATABLE POLITICAL GLOBE,10125_MINI FUNKY DESIGN TAPES,10135_COLOURING PENCILS BROWN TUBE,11001_ASSTD DESIGN RACING CAR PEN,15034_PAPER POCKET TRAVELING FAN,15036_ASSORTED COLOURS SILK FAN,15039_SANDALWOOD FAN,15044A_PINK PAPER PARASOL,15044B_BLUE PAPER PARASOL,15044D_RED PAPER PARASOL,...,90161D_ANT COPPER PINK BOUDICCA BRACELET,90170_DAISY HAIR BAND,90173_PAIR BUTTERFLY HAIR CLIPS,90201A_PURPLE ENAMEL FLOWER RING,90201B_BLACK ENAMEL FLOWER RING,90201C_RED ENAMEL FLOWER RING,90201D_GREEN ENAMEL FLOWER RING,90202D_PINK ENAMEL FLOWER HAIR TIE,90204_PAIR OF ENAMEL BUTTERFLY HAIRCLIP,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
retail_germany.columns = retail_germany.columns.str.replace(" ", "_").str.replace(".", "_").str.replace("-", "_")


retail_germany.columns = retail_germany.columns.str.replace(",", "_").str.replace("/", "_")



retail_germany.columns

Index(['10002_INFLATABLE_POLITICAL_GLOBE_', '10125_MINI_FUNKY_DESIGN_TAPES',
       '10135_COLOURING_PENCILS_BROWN_TUBE',
       '11001_ASSTD_DESIGN_RACING_CAR_PEN',
       '15034_PAPER_POCKET_TRAVELING_FAN_', '15036_ASSORTED_COLOURS_SILK_FAN',
       '15039_SANDALWOOD_FAN', '15044A_PINK_PAPER_PARASOL_',
       '15044B_BLUE_PAPER_PARASOL_', '15044D_RED_PAPER_PARASOL',
       ...
       '90161D_ANT_COPPER_PINK_BOUDICCA_BRACELET', '90170_DAISY_HAIR_BAND',
       '90173_PAIR_BUTTERFLY_HAIR_CLIPS', '90201A_PURPLE_ENAMEL_FLOWER_RING',
       '90201B_BLACK_ENAMEL_FLOWER_RING', '90201C_RED_ENAMEL_FLOWER_RING',
       '90201D_GREEN_ENAMEL_FLOWER_RING', '90202D_PINK_ENAMEL_FLOWER_HAIR_TIE',
       '90204_PAIR_OF_ENAMEL_BUTTERFLY_HAIRCLIP', 'M_Manual'],
      dtype='object', name='item', length=1696)

##### Creating recemendation

In [18]:
frequent_items_germany = apriori(retail_germany, min_support=0.10 , use_colnames=True)


frequent_items_germany

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.102450,(20712_JUMBO_BAG_WOODLAND_ANIMALS)
1,0.129176,(20719_WOODLAND_CHARLOTTE_BAG)
2,0.104677,(22077_6_RIBBONS_RUSTIC_CHARM)
3,0.249443,(22326_ROUND_SNACK_BOXES_SET_OF4_WOODLAND_)
4,0.160356,(22328_ROUND_SNACK_BOXES_SET_OF_4_FRUITS_)
5,0.140312,(22423_REGENCY_CAKESTAND_3_TIER)
6,0.109131,(22551_PLASTERS_IN_TIN_SPACEBOY)
7,0.140312,(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS)
8,0.118040,(22556_PLASTERS_IN_TIN_CIRCUS_PARADE_)
9,0.104677,(22629_SPACEBOY_LUNCH_BOX_)


In [19]:
frequent_titems_germany_combo = association_rules(frequent_items_germany, metric='confidence', min_threshold=0.10)


frequent_titems_germany_combo.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(22328_ROUND_SNACK_BOXES_SET_OF_4_FRUITS_),(22326_ROUND_SNACK_BOXES_SET_OF4_WOODLAND_),0.160356,0.249443,0.13363,0.833333,3.340774,0.09363,4.503341,0.834483
1,(22326_ROUND_SNACK_BOXES_SET_OF4_WOODLAND_),(22328_ROUND_SNACK_BOXES_SET_OF_4_FRUITS_),0.249443,0.160356,0.13363,0.535714,3.340774,0.09363,1.808463,0.933531


# 2. For France

In [20]:
online_retail_france = online_retail[online_retail.Country == 'France']


online_retail_france.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France,22728_ALARM CLOCK BAKELIKE PINK
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France,22727_ALARM CLOCK BAKELIKE RED
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France,22726_ALARM CLOCK BAKELIKE GREEN
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France,21724_PANDA AND BUNNIES STICKER SHEET
4,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France,21883_STARS GIFT TAPE


In [21]:
online_retail_france = online_retail_france.pivot_table(index="InvoiceNo", 
                                                        columns="item", values="CustomerID", aggfunc='count').fillna(0)

online_retail_france.head()

item,10002_INFLATABLE POLITICAL GLOBE,10120_DOGGY RUBBER,10125_MINI FUNKY DESIGN TAPES,10135_COLOURING PENCILS BROWN TUBE,11001_ASSTD DESIGN RACING CAR PEN,15036_ASSORTED COLOURS SILK FAN,15039_SANDALWOOD FAN,15044C_PURPLE PAPER PARASOL,15056BL_EDWARDIAN PARASOL BLACK,15056N_EDWARDIAN PARASOL NATURAL,...,90030B_RED KUKUI COCONUT SEED NECKLACE,90030C_BROWN KUKUI COCONUT SEED NECKLACE,90031_BILI NUT AND WOOD NECKLACE,90099_NECKLACE+BRACELET SET BLUE HIBISCUS,90184B_AMETHYST CHUNKY BEAD BRACELET W STR,90184C_BLACK CHUNKY BEAD BRACELET W STRAP,90201B_BLACK ENAMEL FLOWER RING,90201C_RED ENAMEL FLOWER RING,C2_CARRIAGE,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
retail_france = online_retail_france.applymap(encode)

retail_france.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\2748489437.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_france = online_retail_france.applymap(encode)


item,10002_INFLATABLE POLITICAL GLOBE,10120_DOGGY RUBBER,10125_MINI FUNKY DESIGN TAPES,10135_COLOURING PENCILS BROWN TUBE,11001_ASSTD DESIGN RACING CAR PEN,15036_ASSORTED COLOURS SILK FAN,15039_SANDALWOOD FAN,15044C_PURPLE PAPER PARASOL,15056BL_EDWARDIAN PARASOL BLACK,15056N_EDWARDIAN PARASOL NATURAL,...,90030B_RED KUKUI COCONUT SEED NECKLACE,90030C_BROWN KUKUI COCONUT SEED NECKLACE,90031_BILI NUT AND WOOD NECKLACE,90099_NECKLACE+BRACELET SET BLUE HIBISCUS,90184B_AMETHYST CHUNKY BEAD BRACELET W STR,90184C_BLACK CHUNKY BEAD BRACELET W STRAP,90201B_BLACK ENAMEL FLOWER RING,90201C_RED ENAMEL FLOWER RING,C2_CARRIAGE,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
retail_france.columns = retail_france.columns.str.replace(" ", "_").str.replace(".", "_").str.replace(",", "_")


retail_france.columns = retail_france.columns.str.replace("/", "_").str.replace("-", "_")

###### Creating recommendation

In [24]:
frequent_titems_france = apriori(retail_france, use_colnames=True,min_support=0.10)


frequent_titems_france.head()

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.155039,(20725_LUNCH_BAG_RED_RETROSPOT)
1,0.118863,(20726_LUNCH_BAG_WOODLAND)
2,0.139535,(20750_RED_RETROSPOT_MINI_CASES)
3,0.134367,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_)
4,0.139535,(21086_SET_6_RED_SPOTTY_PAPER_CUPS)


In [25]:
frequent_titems_france_combo = association_rules(frequent_titems_france, metric='confidence',min_threshold=0.1)


frequent_titems_france_combo

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),(21086_SET_6_RED_SPOTTY_PAPER_CUPS),0.134367,0.139535,0.103359,0.769231,5.512821,0.084610,3.728682,0.945672
1,(21086_SET_6_RED_SPOTTY_PAPER_CUPS),(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),0.139535,0.134367,0.103359,0.740741,5.512821,0.084610,3.338870,0.951351
2,(21094_SET_6_RED_SPOTTY_PAPER_PLATES),(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),0.129199,0.134367,0.103359,0.800000,5.953846,0.085999,4.328165,0.955490
3,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),(21094_SET_6_RED_SPOTTY_PAPER_PLATES),0.134367,0.129199,0.103359,0.769231,5.953846,0.085999,3.773471,0.961194
4,(21094_SET_6_RED_SPOTTY_PAPER_PLATES),(21086_SET_6_RED_SPOTTY_PAPER_CUPS),0.129199,0.139535,0.124031,0.960000,6.880000,0.106003,21.511628,0.981454
5,(21086_SET_6_RED_SPOTTY_PAPER_CUPS),(21094_SET_6_RED_SPOTTY_PAPER_PLATES),0.139535,0.129199,0.124031,0.888889,6.880000,0.106003,7.837209,0.993243
6,(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),(22551_PLASTERS_IN_TIN_SPACEBOY),0.173127,0.136951,0.105943,0.611940,4.468319,0.082233,2.224011,0.938720
7,(22551_PLASTERS_IN_TIN_SPACEBOY),(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),0.136951,0.173127,0.105943,0.773585,4.468319,0.082233,3.652024,0.899372
8,(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),(22556_PLASTERS_IN_TIN_CIRCUS_PARADE_),0.173127,0.170543,0.103359,0.597015,3.500678,0.073834,2.058283,0.863906
9,(22556_PLASTERS_IN_TIN_CIRCUS_PARADE_),(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),0.170543,0.173127,0.103359,0.606061,3.500678,0.073834,2.098986,0.861215


# 3. For Australia

In [26]:
online_retail_australia = online_retail[online_retail.Country == 'Australia']


online_retail_australia.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
19,536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,2010-12-01 10:03:00,8.50,12431.0,Australia,22941_CHRISTMAS LIGHTS 10 REINDEER
20,536389,21622,VINTAGE UNION JACK CUSHION COVER,8,2010-12-01 10:03:00,4.95,12431.0,Australia,21622_VINTAGE UNION JACK CUSHION COVER
21,536389,21791,VINTAGE HEADS AND TAILS CARD GAME,12,2010-12-01 10:03:00,1.25,12431.0,Australia,21791_VINTAGE HEADS AND TAILS CARD GAME
22,536389,35004C,SET OF 3 COLOURED FLYING DUCKS,6,2010-12-01 10:03:00,5.45,12431.0,Australia,35004C_SET OF 3 COLOURED FLYING DUCKS
23,536389,35004G,SET OF 3 GOLD FLYING DUCKS,4,2010-12-01 10:03:00,6.35,12431.0,Australia,35004G_SET OF 3 GOLD FLYING DUCKS


In [27]:
retail_australia = online_retail_australia.pivot_table(index="InvoiceNo", 
                                                        columns="item", values="CustomerID", aggfunc='count').fillna(0)

retail_australia.head()

item,15036_ASSORTED COLOURS SILK FAN,15056BL_EDWARDIAN PARASOL BLACK,16161P_WRAP ENGLISH ROSE,16169E_WRAP 50'S CHRISTMAS,20665_RED RETROSPOT PURSE,20675_BLUE POLKADOT BOWL,20676_RED RETROSPOT BOWL,20677_PINK POLKADOT BOWL,20685_DOORMAT RED RETROSPOT,20711_JUMBO BAG TOYS,...,85071B_RED CHARLIE+LOLA PERSONAL DOORSIGN,"85071C_CHARLIE+LOLA""EXTREMELY BUSY"" SIGN",85099B_JUMBO BAG RED RETROSPOT,85099C_JUMBO BAG BAROQUE BLACK WHITE,85114B_IVORY ENCHANTED FOREST PLACEMAT,85114C_RED ENCHANTED FOREST PLACEMAT,85123A_WHITE HANGING HEART T-LIGHT HOLDER,85132A_CHARLIE + LOLA BISCUITS TINS,85152_HAND OVER THE CHOCOLATE SIGN,85183B_CHARLIE & LOLA WASTEPAPER BIN FLORA
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540267,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
retail_australia = retail_australia.applymap(encode)

retail_australia.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\2389758669.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_australia = retail_australia.applymap(encode)


item,15036_ASSORTED COLOURS SILK FAN,15056BL_EDWARDIAN PARASOL BLACK,16161P_WRAP ENGLISH ROSE,16169E_WRAP 50'S CHRISTMAS,20665_RED RETROSPOT PURSE,20675_BLUE POLKADOT BOWL,20676_RED RETROSPOT BOWL,20677_PINK POLKADOT BOWL,20685_DOORMAT RED RETROSPOT,20711_JUMBO BAG TOYS,...,85071B_RED CHARLIE+LOLA PERSONAL DOORSIGN,"85071C_CHARLIE+LOLA""EXTREMELY BUSY"" SIGN",85099B_JUMBO BAG RED RETROSPOT,85099C_JUMBO BAG BAROQUE BLACK WHITE,85114B_IVORY ENCHANTED FOREST PLACEMAT,85114C_RED ENCHANTED FOREST PLACEMAT,85123A_WHITE HANGING HEART T-LIGHT HOLDER,85132A_CHARLIE + LOLA BISCUITS TINS,85152_HAND OVER THE CHOCOLATE SIGN,85183B_CHARLIE & LOLA WASTEPAPER BIN FLORA
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537676,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539419,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
540267,0,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
540280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
retail_australia.columns = retail_australia.columns.str.replace(" ", "_").str.replace(".", "_").str.replace(",", "_")


retail_australia.columns = retail_australia.columns.str.replace("/", "_").str.replace("-", "_")

###### Creating recommendation

In [30]:
frequent_titems_australia = apriori(retail_australia, use_colnames=True,min_support=0.10)


frequent_titems_australia.head()

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.142857,(20725_LUNCH_BAG_RED_RETROSPOT)
1,0.107143,(20726_LUNCH_BAG_WOODLAND)
2,0.160714,(21731_RED_TOADSTOOL_LED_NIGHT_LIGHT)
3,0.125000,(21915_RED__HARMONICA_IN_BOX_)
4,0.125000,(22090_PAPER_BUNTING_RETROSPOT)


In [31]:
frequent_titems_australia_combo = association_rules(frequent_titems_australia, metric='confidence',min_threshold=0.1)


frequent_titems_australia_combo.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(22629_SPACEBOY_LUNCH_BOX_),(22630_DOLLY_GIRL_LUNCH_BOX),0.107143,0.107143,0.107143,1.0,9.333333,0.095663,inf,1.0
1,(22630_DOLLY_GIRL_LUNCH_BOX),(22629_SPACEBOY_LUNCH_BOX_),0.107143,0.107143,0.107143,1.0,9.333333,0.095663,inf,1.0
2,(22726_ALARM_CLOCK_BAKELIKE_GREEN),(22727_ALARM_CLOCK_BAKELIKE_RED_),0.107143,0.107143,0.107143,1.0,9.333333,0.095663,inf,1.0
3,(22727_ALARM_CLOCK_BAKELIKE_RED_),(22726_ALARM_CLOCK_BAKELIKE_GREEN),0.107143,0.107143,0.107143,1.0,9.333333,0.095663,inf,1.0


# 4. For Singapore

In [32]:
online_retail_singapore = online_retail[online_retail.Country == 'Singapore']


online_retail_singapore.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
2081,542102,21519,GIN & TONIC DIET GREETING CARD,72,2011-01-25 13:26:00,0.36,12744.0,Singapore,21519_GIN & TONIC DIET GREETING CARD
2082,542102,22697,GREEN REGENCY TEACUP AND SAUCER,6,2011-01-25 13:26:00,2.95,12744.0,Singapore,22697_GREEN REGENCY TEACUP AND SAUCER
2083,542102,22699,ROSES REGENCY TEACUP AND SAUCER,6,2011-01-25 13:26:00,2.95,12744.0,Singapore,22699_ROSES REGENCY TEACUP AND SAUCER
2084,542102,22343,PARTY PIZZA DISH RED RETROSPOT,24,2011-01-25 13:26:00,0.21,12744.0,Singapore,22343_PARTY PIZZA DISH RED RETROSPOT
2085,542102,22344,PARTY PIZZA DISH PINK POLKADOT,24,2011-01-25 13:26:00,0.21,12744.0,Singapore,22344_PARTY PIZZA DISH PINK POLKADOT


In [33]:
retail_singapore = online_retail_singapore.pivot_table(index="InvoiceNo", 
                                                        columns="item", values="CustomerID", aggfunc='count').fillna(0)

retail_singapore.head()

item,20681_PINK POLKADOT CHILDRENS UMBRELLA,20682_RED RETROSPOT CHILDRENS UMBRELLA,20685_DOORMAT RED RETROSPOT,20754_RETROSPOT RED WASHING UP GLOVES,20971_PINK BLUE FELT CRAFT TRINKET BOX,20972_PINK CREAM FELT CRAFT TRINKET BOX,20984_12 PENCILS TALL TUBE POSY,21086_SET/6 RED SPOTTY PAPER CUPS,21094_SET/6 RED SPOTTY PAPER PLATES,21156_RETROSPOT CHILDRENS APRON,...,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85048_15CM CHRISTMAS GLASS BALL 20 LIGHTS,85071A_BLUE CHARLIE+LOLA PERSONAL DOORSIGN,85099B_JUMBO BAG RED RETROSPOT,85123A_WHITE HANGING HEART T-LIGHT HOLDER,85150_LADIES & GENTLEMEN METAL SIGN,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
542102,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
548813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
548820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
548968,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
560360,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [34]:
retail_singapore

item,20681_PINK POLKADOT CHILDRENS UMBRELLA,20682_RED RETROSPOT CHILDRENS UMBRELLA,20685_DOORMAT RED RETROSPOT,20754_RETROSPOT RED WASHING UP GLOVES,20971_PINK BLUE FELT CRAFT TRINKET BOX,20972_PINK CREAM FELT CRAFT TRINKET BOX,20984_12 PENCILS TALL TUBE POSY,21086_SET/6 RED SPOTTY PAPER CUPS,21094_SET/6 RED SPOTTY PAPER PLATES,21156_RETROSPOT CHILDRENS APRON,...,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85048_15CM CHRISTMAS GLASS BALL 20 LIGHTS,85071A_BLUE CHARLIE+LOLA PERSONAL DOORSIGN,85099B_JUMBO BAG RED RETROSPOT,85123A_WHITE HANGING HEART T-LIGHT HOLDER,85150_LADIES & GENTLEMEN METAL SIGN,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
542102,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
548813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
548820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
548968,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
560360,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
571239,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
571751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [35]:
retail_singapore = retail_singapore.applymap(encode)

retail_singapore.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\307738131.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_singapore = retail_singapore.applymap(encode)


item,20681_PINK POLKADOT CHILDRENS UMBRELLA,20682_RED RETROSPOT CHILDRENS UMBRELLA,20685_DOORMAT RED RETROSPOT,20754_RETROSPOT RED WASHING UP GLOVES,20971_PINK BLUE FELT CRAFT TRINKET BOX,20972_PINK CREAM FELT CRAFT TRINKET BOX,20984_12 PENCILS TALL TUBE POSY,21086_SET/6 RED SPOTTY PAPER CUPS,21094_SET/6 RED SPOTTY PAPER PLATES,21156_RETROSPOT CHILDRENS APRON,...,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85048_15CM CHRISTMAS GLASS BALL 20 LIGHTS,85071A_BLUE CHARLIE+LOLA PERSONAL DOORSIGN,85099B_JUMBO BAG RED RETROSPOT,85123A_WHITE HANGING HEART T-LIGHT HOLDER,85150_LADIES & GENTLEMEN METAL SIGN,M_Manual
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
542102,1,1,1,0,1,1,0,1,1,1,...,0,0,0,0,0,0,1,1,1,0
548813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
548820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
548968,0,0,1,1,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,1,0
560360,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [36]:
retail_singapore.columns = retail_singapore.columns.str.replace(" ", "_").str.replace(".", "_").str.replace(",", "_")


retail_singapore.columns = retail_singapore.columns.str.replace("/", "_").str.replace("-", "_")

###### Creating recommendation

In [37]:
frequent_titems_singapore = apriori(retail_singapore, use_colnames=True,min_support=0.4)


frequent_titems_singapore.head()

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.428571,(20685_DOORMAT_RED_RETROSPOT)
1,0.428571,(21181_PLEASE_ONE_PERSON_METAL_SIGN)
2,0.571429,(22343_PARTY_PIZZA_DISH_RED_RETROSPOT)
3,0.428571,(48138_DOORMAT_UNION_FLAG)
4,0.428571,(85150_LADIES_&_GENTLEMEN_METAL_SIGN)


In [38]:
frequent_titems_singapore_combo = association_rules(frequent_titems_singapore, metric='confidence',min_threshold=0.9)


frequent_titems_singapore_combo.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(20685_DOORMAT_RED_RETROSPOT),(21181_PLEASE_ONE_PERSON_METAL_SIGN),0.428571,0.428571,0.428571,1.0,2.333333,0.244898,inf,1.00
1,(21181_PLEASE_ONE_PERSON_METAL_SIGN),(20685_DOORMAT_RED_RETROSPOT),0.428571,0.428571,0.428571,1.0,2.333333,0.244898,inf,1.00
2,(20685_DOORMAT_RED_RETROSPOT),(22343_PARTY_PIZZA_DISH_RED_RETROSPOT),0.428571,0.571429,0.428571,1.0,1.750000,0.183673,inf,0.75
3,(20685_DOORMAT_RED_RETROSPOT),(48138_DOORMAT_UNION_FLAG),0.428571,0.428571,0.428571,1.0,2.333333,0.244898,inf,1.00
4,(48138_DOORMAT_UNION_FLAG),(20685_DOORMAT_RED_RETROSPOT),0.428571,0.428571,0.428571,1.0,2.333333,0.244898,inf,1.00


# 5. For USA

In [39]:
online_retail_usa = online_retail[online_retail.Country == 'USA']


online_retail_usa.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
5014,550644,22722,SET OF 6 SPICE TINS PANTRY DESIGN,7,2011-04-19 16:19:00,3.95,12733.0,USA,22722_SET OF 6 SPICE TINS PANTRY DESIGN
5015,550644,22979,PANTRY WASHING UP BRUSH,2,2011-04-19 16:19:00,1.45,12733.0,USA,22979_PANTRY WASHING UP BRUSH
5016,550644,84987,SET OF 36 TEATIME PAPER DOILIES,3,2011-04-19 16:19:00,1.45,12733.0,USA,84987_SET OF 36 TEATIME PAPER DOILIES
5017,550644,22720,SET OF 3 CAKE TINS PANTRY DESIGN,1,2011-04-19 16:19:00,4.95,12733.0,USA,22720_SET OF 3 CAKE TINS PANTRY DESIGN
5018,550644,22993,SET OF 4 PANTRY JELLY MOULDS,1,2011-04-19 16:19:00,1.25,12733.0,USA,22993_SET OF 4 PANTRY JELLY MOULDS


In [40]:
retail_usa = online_retail_usa.pivot_table(index="InvoiceNo", 
                                                        columns="item", values="CustomerID", aggfunc='count').fillna(0)

retail_usa.head()

item,16161P_WRAP ENGLISH ROSE,20674_GREEN POLKADOT BOWL,20675_BLUE POLKADOT BOWL,20676_RED RETROSPOT BOWL,20677_PINK POLKADOT BOWL,20750_RED RETROSPOT MINI CASES,20975_12 PENCILS SMALL TUBE RED RETROSPOT,21058_PARTY INVITES WOODLAND,21059_PARTY INVITES DINOSAURS,21067_VINTAGE RED TEATIME MUG,...,79191C_RETRO PLASTIC ELEPHANT TRAY,84509A_SET OF 4 ENGLISH ROSE PLACEMATS,84828_JUNGLE POPSICLES ICE LOLLY MOULDS,84987_SET OF 36 TEATIME PAPER DOILIES,84991_60 TEATIME FAIRY CAKE CASES,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85099B_JUMBO BAG RED RETROSPOT
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
550644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
570467,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
572215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
580553,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [41]:
retail_usa

item,16161P_WRAP ENGLISH ROSE,20674_GREEN POLKADOT BOWL,20675_BLUE POLKADOT BOWL,20676_RED RETROSPOT BOWL,20677_PINK POLKADOT BOWL,20750_RED RETROSPOT MINI CASES,20975_12 PENCILS SMALL TUBE RED RETROSPOT,21058_PARTY INVITES WOODLAND,21059_PARTY INVITES DINOSAURS,21067_VINTAGE RED TEATIME MUG,...,79191C_RETRO PLASTIC ELEPHANT TRAY,84509A_SET OF 4 ENGLISH ROSE PLACEMATS,84828_JUNGLE POPSICLES ICE LOLLY MOULDS,84987_SET OF 36 TEATIME PAPER DOILIES,84991_60 TEATIME FAIRY CAKE CASES,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85099B_JUMBO BAG RED RETROSPOT
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
550644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
570467,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
572215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
580553,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [42]:
retail_usa = retail_usa.applymap(encode)

retail_usa.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\3146481560.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_usa = retail_usa.applymap(encode)


item,16161P_WRAP ENGLISH ROSE,20674_GREEN POLKADOT BOWL,20675_BLUE POLKADOT BOWL,20676_RED RETROSPOT BOWL,20677_PINK POLKADOT BOWL,20750_RED RETROSPOT MINI CASES,20975_12 PENCILS SMALL TUBE RED RETROSPOT,21058_PARTY INVITES WOODLAND,21059_PARTY INVITES DINOSAURS,21067_VINTAGE RED TEATIME MUG,...,79191C_RETRO PLASTIC ELEPHANT TRAY,84509A_SET OF 4 ENGLISH ROSE PLACEMATS,84828_JUNGLE POPSICLES ICE LOLLY MOULDS,84987_SET OF 36 TEATIME PAPER DOILIES,84991_60 TEATIME FAIRY CAKE CASES,84997A_CHILDRENS CUTLERY POLKADOT GREEN,84997B_CHILDRENS CUTLERY RETROSPOT RED,84997C_CHILDRENS CUTLERY POLKADOT BLUE,84997D_CHILDRENS CUTLERY POLKADOT PINK,85099B_JUMBO BAG RED RETROSPOT
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
550644,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
570467,1,0,0,0,0,1,0,1,1,0,...,0,1,0,1,1,0,0,0,0,0
572215,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
580158,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
580553,0,1,1,1,1,0,1,0,0,0,...,0,0,0,0,0,1,1,1,1,0


In [43]:
retail_usa.columns = retail_usa.columns.str.replace(" ", "_").str.replace(".", "_").str.replace(",", "_")


retail_usa.columns = retail_usa.columns.str.replace("/", "_").str.replace("-", "_")

###### Creating recommendation

In [44]:
frequent_titems_usa = apriori(retail_usa, use_colnames=True,min_support=0.4)


frequent_titems_usa.head()

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.4,(21121_SET_10_RED_POLKADOT_PARTY_CANDLES)
1,0.4,(21122_SET_10_PINK_POLKADOT_PARTY_CANDLES)
2,0.4,(21123_SET_10_IVORY_POLKADOT_PARTY_CANDLES)
3,0.4,(21124_SET_10_BLUE_POLKADOT_PARTY_CANDLES)
4,0.4,(22027_TEA_PARTY_BIRTHDAY_CARD)


In [45]:
frequent_titems_usa_combo = association_rules(frequent_titems_usa, metric='confidence',min_threshold=0.8)


frequent_titems_usa_combo.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),(21122_SET_10_PINK_POLKADOT_PARTY_CANDLES),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
1,(21122_SET_10_PINK_POLKADOT_PARTY_CANDLES),(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
2,(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),(21123_SET_10_IVORY_POLKADOT_PARTY_CANDLES),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
3,(21123_SET_10_IVORY_POLKADOT_PARTY_CANDLES),(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
4,(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),(21124_SET_10_BLUE_POLKADOT_PARTY_CANDLES),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0


# 5. For Canada

In [46]:
online_retail_canada = online_retail[online_retail.Country == 'Canada']


online_retail_canada.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item
3714,546533,20886,BOX OF 9 PEBBLE CANDLES,12,2011-03-14 13:53:00,1.95,15388.0,Canada,20886_BOX OF 9 PEBBLE CANDLES
3715,546533,79030D,"TUMBLER, BAROQUE",6,2011-03-14 13:53:00,1.65,15388.0,Canada,"79030D_TUMBLER, BAROQUE"
3716,546533,21132,SILVER STANDING GNOME,4,2011-03-14 13:53:00,4.25,15388.0,Canada,21132_SILVER STANDING GNOME
3717,546533,84879,ASSORTED COLOUR BIRD ORNAMENT,8,2011-03-14 13:53:00,1.69,15388.0,Canada,84879_ASSORTED COLOUR BIRD ORNAMENT
3718,546533,84755,COLOUR GLASS T-LIGHT HOLDER HANGING,16,2011-03-14 13:53:00,0.65,15388.0,Canada,84755_COLOUR GLASS T-LIGHT HOLDER HANGING


In [47]:
retail_canada = online_retail_canada.pivot_table(index="InvoiceNo", 
                                                        columns="item", values="CustomerID", aggfunc='count').fillna(0)

retail_canada.head()

item,10133_COLOURING PENCILS BROWN TUBE,10135_COLOURING PENCILS BROWN TUBE,15044A_PINK PAPER PARASOL,15044C_PURPLE PAPER PARASOL,15044D_RED PAPER PARASOL,16156L_WRAP CAROUSEL,16161G_WRAP BAD HAIR DAY,16161P_WRAP ENGLISH ROSE,20726_LUNCH BAG WOODLAND,20727_LUNCH BAG BLACK SKULL.,...,84879_ASSORTED COLOUR BIRD ORNAMENT,84926A_WAKE UP COCKEREL TILE COASTER,84926D_LA PALMIERA TILE COASTER,85116_BLACK CANDELABRA T-LIGHT HOLDER,"85159A_BLACK TEA,COFFEE,SUGAR JARS","85159B_WHITE TEA,COFFEE,SUGAR JARS",85163B_BLACK BAROQUE WALL CLOCK,85168B_BLACK BAROQUE CARRIAGE CLOCK,85176_SEWING SUSAN 21 NEEDLE SET,85232D_SET/3 DECOUPAGE STACKING TINS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
546533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
551736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557322,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
559557,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
564428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
retail_canada

item,10133_COLOURING PENCILS BROWN TUBE,10135_COLOURING PENCILS BROWN TUBE,15044A_PINK PAPER PARASOL,15044C_PURPLE PAPER PARASOL,15044D_RED PAPER PARASOL,16156L_WRAP CAROUSEL,16161G_WRAP BAD HAIR DAY,16161P_WRAP ENGLISH ROSE,20726_LUNCH BAG WOODLAND,20727_LUNCH BAG BLACK SKULL.,...,84879_ASSORTED COLOUR BIRD ORNAMENT,84926A_WAKE UP COCKEREL TILE COASTER,84926D_LA PALMIERA TILE COASTER,85116_BLACK CANDELABRA T-LIGHT HOLDER,"85159A_BLACK TEA,COFFEE,SUGAR JARS","85159B_WHITE TEA,COFFEE,SUGAR JARS",85163B_BLACK BAROQUE WALL CLOCK,85168B_BLACK BAROQUE CARRIAGE CLOCK,85176_SEWING SUSAN 21 NEEDLE SET,85232D_SET/3 DECOUPAGE STACKING TINS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
546533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
551736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557322,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
559557,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
564428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
retail_canada = retail_canada.applymap(encode)

retail_canada.head()

C:\Users\navee\AppData\Local\Temp\ipykernel_15188\880986804.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  retail_canada = retail_canada.applymap(encode)


item,10133_COLOURING PENCILS BROWN TUBE,10135_COLOURING PENCILS BROWN TUBE,15044A_PINK PAPER PARASOL,15044C_PURPLE PAPER PARASOL,15044D_RED PAPER PARASOL,16156L_WRAP CAROUSEL,16161G_WRAP BAD HAIR DAY,16161P_WRAP ENGLISH ROSE,20726_LUNCH BAG WOODLAND,20727_LUNCH BAG BLACK SKULL.,...,84879_ASSORTED COLOUR BIRD ORNAMENT,84926A_WAKE UP COCKEREL TILE COASTER,84926D_LA PALMIERA TILE COASTER,85116_BLACK CANDELABRA T-LIGHT HOLDER,"85159A_BLACK TEA,COFFEE,SUGAR JARS","85159B_WHITE TEA,COFFEE,SUGAR JARS",85163B_BLACK BAROQUE WALL CLOCK,85168B_BLACK BAROQUE CARRIAGE CLOCK,85176_SEWING SUSAN 21 NEEDLE SET,85232D_SET/3 DECOUPAGE STACKING TINS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
546533,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
551736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
557322,1,1,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,1,1,1,1
559557,1,0,1,1,1,0,0,0,1,0,...,0,1,1,0,1,1,0,0,0,0
564428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
retail_canada.columns = retail_canada.columns.str.replace(" ", "_").str.replace(".", "_").str.replace(",", "_")


retail_canada.columns = retail_canada.columns.str.replace("/", "_").str.replace("-", "_")

###### Creating recommendation

In [51]:
frequent_titems_canada = apriori(retail_canada, use_colnames=True,min_support=0.4)


frequent_titems_canada.head()

C:\Users\navee\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.4,(10133_COLOURING_PENCILS_BROWN_TUBE)
1,0.4,(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__)
2,0.4,(23192_BUNDLE_OF_3_ALPHABET_EXERCISE_BOOKS)
3,0.4,"(10133_COLOURING_PENCILS_BROWN_TUBE, 23190_BUN..."
4,0.4,"(10133_COLOURING_PENCILS_BROWN_TUBE, 23192_BUN..."


In [52]:
frequent_titems_canada_combo = association_rules(frequent_titems_canada, metric='confidence',min_threshold=0.8)


frequent_titems_canada_combo.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(10133_COLOURING_PENCILS_BROWN_TUBE),(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
1,(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__),(10133_COLOURING_PENCILS_BROWN_TUBE),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
2,(10133_COLOURING_PENCILS_BROWN_TUBE),(23192_BUNDLE_OF_3_ALPHABET_EXERCISE_BOOKS),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
3,(23192_BUNDLE_OF_3_ALPHABET_EXERCISE_BOOKS),(10133_COLOURING_PENCILS_BROWN_TUBE),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
4,(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__),(23192_BUNDLE_OF_3_ALPHABET_EXERCISE_BOOKS),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0


In [53]:
frequent_titems_canada

,support,itemsets
0,0.4,(10133_COLOURING_PENCILS_BROWN_TUBE)
1,0.4,(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__)
2,0.4,(23192_BUNDLE_OF_3_ALPHABET_EXERCISE_BOOKS)
3,0.4,"(10133_COLOURING_PENCILS_BROWN_TUBE, 23190_BUN..."
4,0.4,"(10133_COLOURING_PENCILS_BROWN_TUBE, 23192_BUN..."
5,0.4,"(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__, 23..."
6,0.4,"(10133_COLOURING_PENCILS_BROWN_TUBE, 23190_BUN..."


In [60]:
frequent_titems_france_combo

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),(21086_SET_6_RED_SPOTTY_PAPER_CUPS),0.134367,0.139535,0.103359,0.769231,5.512821,0.084610,3.728682,0.945672
1,(21086_SET_6_RED_SPOTTY_PAPER_CUPS),(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),0.139535,0.134367,0.103359,0.740741,5.512821,0.084610,3.338870,0.951351
2,(21094_SET_6_RED_SPOTTY_PAPER_PLATES),(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),0.129199,0.134367,0.103359,0.800000,5.953846,0.085999,4.328165,0.955490
3,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),(21094_SET_6_RED_SPOTTY_PAPER_PLATES),0.134367,0.129199,0.103359,0.769231,5.953846,0.085999,3.773471,0.961194
4,(21094_SET_6_RED_SPOTTY_PAPER_PLATES),(21086_SET_6_RED_SPOTTY_PAPER_CUPS),0.129199,0.139535,0.124031,0.960000,6.880000,0.106003,21.511628,0.981454
5,(21086_SET_6_RED_SPOTTY_PAPER_CUPS),(21094_SET_6_RED_SPOTTY_PAPER_PLATES),0.139535,0.129199,0.124031,0.888889,6.880000,0.106003,7.837209,0.993243
6,(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),(22551_PLASTERS_IN_TIN_SPACEBOY),0.173127,0.136951,0.105943,0.611940,4.468319,0.082233,2.224011,0.938720
7,(22551_PLASTERS_IN_TIN_SPACEBOY),(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),0.136951,0.173127,0.105943,0.773585,4.468319,0.082233,3.652024,0.899372
8,(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),(22556_PLASTERS_IN_TIN_CIRCUS_PARADE_),0.173127,0.170543,0.103359,0.597015,3.500678,0.073834,2.058283,0.863906
9,(22556_PLASTERS_IN_TIN_CIRCUS_PARADE_),(22554_PLASTERS_IN_TIN_WOODLAND_ANIMALS),0.170543,0.173127,0.103359,0.606061,3.500678,0.073834,2.098986,0.861215


# Best Products to sell togather

#### In germarny

In [64]:
frequent_titems_germany_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(22328_ROUND_SNACK_BOXES_SET_OF_4_FRUITS_),(22326_ROUND_SNACK_BOXES_SET_OF4_WOODLAND_)


In [65]:
#### In france

frequent_titems_france_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(21080_SET_20_RED_RETROSPOT_PAPER_NAPKINS_),(21086_SET_6_RED_SPOTTY_PAPER_CUPS)


In [66]:
#### In australia

frequent_titems_australia_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(22629_SPACEBOY_LUNCH_BOX_),(22630_DOLLY_GIRL_LUNCH_BOX)


In [67]:
#### In singapore

frequent_titems_singapore_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(20685_DOORMAT_RED_RETROSPOT),(21181_PLEASE_ONE_PERSON_METAL_SIGN)


In [68]:
#### In usa

frequent_titems_usa_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(21121_SET_10_RED_POLKADOT_PARTY_CANDLES),(21122_SET_10_PINK_POLKADOT_PARTY_CANDLES)


In [69]:
#### In canada

frequent_titems_canada_combo[["antecedents","consequents"]].head(1)

,antecedents,consequents
0,(10133_COLOURING_PENCILS_BROWN_TUBE),(23190_BUNDLE_OF_3_SCHOOL_EXERCISE_BOOKS__)
